In [1]:
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
from tkinter import scrolledtext 

from pyfuseki import FusekiUpdate
from pyfuseki.utils import RdfUtils
from SPARQLWrapper import SPARQLWrapper, JSON, TURTLE
from rdflib import Graph, RDFS, URIRef, Namespace, RDF, Literal

import networkx as nx
from PIL import Image, ImageTk
import matplotlib.pyplot as plt

import copy
import string
import random

Duplicate key in file WindowsPath('C:/Users/wan-h/anaconda3/lib/site-packages/matplotlib/mpl-data/matplotlibrc'), line 258 ('font.family:  sans-serif')


In [2]:
class Application(tk.Frame):
  # treeview Widgetの情報を格納する変数
  treeview = None

  def __init__(self, master=None):
    # Windowの初期設定を行う。
    super().__init__(master)
    self.master = master
    self.master.title("GPM Window")
    self.master.geometry("1200x650")

    self.prefix_grpah_data = {}
    self.ttl_file = 'ttl_file.ttl'
    self.selected_graph = ''
    self.selected_graph_name = ''
    self.gpm_uri = ''
    self.graph_node_list = []
    self.graph_edge_list = []
    self.graph_label_list = {}
    self.graph_position = {}
    # ログウインドウで選択したグラフ
    self.log_graph = tk.StringVar()
    # ログウインドウで選択したノード
    self.log_node = tk.StringVar()
    # ログウインドウで選択したuri
    self.log_uri = tk.StringVar()
    # 表示しているグラフの層を記録
    self.graph_level = ''
    self.create_widgets()

  #------------------------------------------------------------------
  # ウィジェットを作成
  #------------------------------------------------------------------
  def create_widgets(self):
    #------------------------------------------------------------------
    # LabelFrame
    #------------------------------------------------------------------
    self.create_frame_widget()
    #------------------------------------------------------------------
    # listbox widget (Graph Info)
    #------------------------------------------------------------------
    self.scrollbar_ep_info_x = tk.Scrollbar(self.frame_ep_info_list, orient=tk.HORIZONTAL)
    self.scrollbar_ep_info_x.pack(side="bottom", fill="x")
    self.scrollbar_ep_info_y = tk.Scrollbar(self.frame_ep_info_list, orient=tk.VERTICAL)
    self.scrollbar_ep_info_y.pack(side="right", fill="y")

    self.listbox_ep_value = tk.StringVar()
    self.graph_list = get_graph()
    self.listbox_ep_value.set(self.graph_list)
    self.listbox_ep_info = tk.Listbox(self.frame_ep_info_list, height=7, width=30, selectmode=tk.SINGLE,\
                            listvariable=self.listbox_ep_value, \
                            exportselection=False, xscrollcommand=self.scrollbar_ep_info_x.set,\
                            yscrollcommand=self.scrollbar_ep_info_y.set)  
    self.listbox_ep_info.pack(fill="both")
    self.listbox_ep_info.bind("<<ListboxSelect>>", self.select_graph)

    self.scrollbar_ep_info_x.config(command=self.listbox_ep_info.xview)
    self.scrollbar_ep_info_y.config(command=self.listbox_ep_info.yview)
    #------------------------------------------------------------------
    # treeview widget
    #------------------------------------------------------------------
    self.treeview = ttk.Treeview(self.frame_hierarchy_list)
    # 階層列設定
    self.treeview.column("#0", width=180, minwidth=500)
    # 階層列(ツリーカラム)の見出し設定。階層列とする。
    self.treeview.heading("#0")
    self.treeview.bind("<<TreeviewSelect>>", self.tree_select)
    self.treeview.grid(row=1, column=1, sticky="nsew")

    tree_scrollbar_y = ttk.Scrollbar(self.frame_hierarchy_list, orient="vertical",\
                      command=self.treeview.yview)
    tree_scrollbar_y.grid(row=1, column=2, sticky="nsew")
    self.treeview.configure(yscrollcommand=tree_scrollbar_y.set)

    tree_scrollbar_x = ttk.Scrollbar(self.frame_hierarchy_list, orient="horizontal",\
                      command=self.treeview.xview)
    tree_scrollbar_x.grid(row=2, column=1, sticky="nsew")
    self.treeview.configure(xscrollcommand=tree_scrollbar_x.set)
    #------------------------------------------------------------------
    # ノード情報ウィジェットを作成
    #------------------------------------------------------------------
    self.create_node_info_widget()
    #------------------------------------------------------------------
    # listbox widget (GPM uri Info)
    #------------------------------------------------------------------
    self.scrollbar_gpm_uri_list_x = tk.Scrollbar(self.frame_new_gpm_list, orient=tk.HORIZONTAL)
    self.scrollbar_gpm_uri_list_x.pack(side="bottom", fill="x")
    self.scrollbar_gpm_uri_list_y = tk.Scrollbar(self.frame_new_gpm_list, orient=tk.VERTICAL)
    self.scrollbar_gpm_uri_list_y.pack(side="right", fill="y")

    self.listbox_gpm_uri_list = tk.Listbox(self.frame_new_gpm_list, height=7, width=30, selectmode=tk.SINGLE,\
                            exportselection=False, xscrollcommand=self.scrollbar_gpm_uri_list_x.set,\
                            yscrollcommand=self.scrollbar_gpm_uri_list_y.set)  
    self.listbox_gpm_uri_list.pack(fill="both")

    self.scrollbar_gpm_uri_list_x.config(command=self.listbox_gpm_uri_list.xview)
    self.scrollbar_gpm_uri_list_y.config(command=self.listbox_gpm_uri_list.yview)
    #------------------------------------------------------------------
    # listbox widget (container ep list)
    #------------------------------------------------------------------
    self.scrollbar_con_ep_x = tk.Scrollbar(self.frame_con_ep_list, orient=tk.HORIZONTAL)
    self.scrollbar_con_ep_x.pack(side="bottom", fill="x")
    self.scrollbar_con_ep_y = tk.Scrollbar(self.frame_con_ep_list, orient=tk.VERTICAL)
    self.scrollbar_con_ep_y.pack(side="right", fill="y")

    self.list_con_ep = tk.Listbox(self.frame_con_ep_list, height=7, \
                            selectmode=tk.SINGLE, exportselection=False, \
                            xscrollcommand=self.scrollbar_con_ep_x.set,\
                            yscrollcommand=self.scrollbar_con_ep_y.set)
    self.list_con_ep.pack(fill="both")

    self.scrollbar_con_ep_x.config(command=self.list_con_ep.xview)
    self.scrollbar_con_ep_y.config(command=self.list_con_ep.yview)
    #------------------------------------------------------------------
    # Button widget
    #------------------------------------------------------------------
    # グラフエクスポート
    self.button_export = tk.Button(self.frame_export, text="Fuseki Export (Graph)", width=17, height=2,\
                        command=fuseki_export)
    self.button_export.grid(row=1, column=0, padx=5)
    # オールエクスポート
    self.button_export_all = tk.Button(self.frame_export, text="Fuseki Export (All)", width=15, height=2,\
                        command=fuseki_export_all)
    self.button_export_all.grid(row=1, column=1, padx=5)
    # 表示EPを設定
    self.button_gpm_set = tk.Button(self.frame_button, text=" 設定 ", width=8, height=2, \
                        command=self.set_gpm)
    self.button_gpm_set.grid(row=1, column=0, padx=5)
    # ログウィンドウ開く
    self.button_log_window = tk.Button(self.frame_button, text=" Log Window Open ", width=15, height=2,\
                          command=self.open_log_window)
    self.button_log_window.grid(row=1, column=1, padx=5)
    # Use追加
    self.button_use_insert = tk.Button(self.frame_use_relation, text=" 追加 ", width=7, height=2,\
                        command=self.insert_use_event)
    self.button_use_insert.grid(row=1, column=0, padx=5)
    # Use削除
    self.button_use_delete = tk.Button(self.frame_use_relation, text=" 削除 ", width=7, height=2,\
                        command=self.delete_use_event)
    self.button_use_delete.grid(row=1, column=1, padx=5)
    # コンテナ追加
    self.button_con_insert = tk.Button(self.frame_excute, text=" 追加 ", width=20, height=2, \
                          command=self.insert_member_event)
    self.button_con_insert.grid(row=0, column=0)
    # コンテナ削除
    self.button_con_delete = tk.Button(self.frame_excute, text=" 削除 ", width=20, height=2, \
                          command=self.delete_member_event)
    self.button_con_delete.grid(row=1, column=0)
    #------------------------------------------------------------------
    # ★★★★★★★★★★★
    #------------------------------------------------------------------
    self.label_log_graph = tk.Label(self.frame_button, textvariable=self.log_graph)
    self.label_log_uri = tk.Label(self.frame_button, textvariable=self.log_uri)
    self.label_log_node = tk.Label(self.frame_button, textvariable=self.log_node)
    #------------------------------------------------------------------
    # canvas
    #------------------------------------------------------------------
    self.graph_canvas = tk.Canvas(self.master)
    self.graph_canvas.place(x=0, y=0, width=480, height=570)
    #canvas スクロールバー
    self.canvas_bar_y = tk.Scrollbar(self.graph_canvas, orient=tk.VERTICAL)
    self.canvas_bar_x = tk.Scrollbar(self.graph_canvas, orient=tk.HORIZONTAL)
    self.canvas_bar_y.pack(side=tk.RIGHT, fill=tk.Y)
    self.canvas_bar_x.pack(side=tk.BOTTOM, fill=tk.X)
    self.canvas_bar_y.config(command=self.graph_canvas.yview)
    self.canvas_bar_x.config(command=self.graph_canvas.xview)
    self.graph_canvas.config(yscrollcommand=self.canvas_bar_y.set, xscrollcommand=self.canvas_bar_x.set)
    # Canvasのスクロール範囲を設定
    self.graph_canvas.config(scrollregion=(0, 0, 1000, 1000))

  #------------------------------------------------------------------
  # LabelFrameを作成
  #------------------------------------------------------------------
  def create_frame_widget(self):
    frame_options = {"bd": 2, "relief": "ridge", "foreground": "green"}
    # graph info
    self.frame_ep_info_list = tk.LabelFrame(self.master, text=" Graph Info", width=300, height=120,\
                                            **frame_options)
    self.frame_ep_info_list.place(x=490,y=0)
    # Top Level URI List
    self.frame_new_gpm_list = tk.LabelFrame(self.master, text="Top Level EP List", width=300, height=200,\
                                            **frame_options)
    self.frame_new_gpm_list.place(x=490, y=155)
    # node list
    self.frame_hierarchy_list = tk.LabelFrame(self.master, text="Action List", width=300, height=240,\
                                            **frame_options)
    self.frame_hierarchy_list.place(x=490,y=310)
    # node info
    self.frame_node = tk.LabelFrame(self.master, text="Infromation", width=300, height=300,\
                                    **frame_options)
    self.frame_node.place(x=705, y=0)
    # id
    self.frame_id_info = tk.LabelFrame(self.frame_node, width=300, height=300,\
                                        **frame_options)
    self.frame_id_info.grid(row=0, column=0)
    # action
    self.frame_action_info = tk.LabelFrame(self.frame_node, width=300, height=300,\
                                        **frame_options)
    self.frame_action_info.grid(row=1, column=0)
    # height
    self.frame_height_info = tk.LabelFrame(self.frame_node, width=300, height=300,\
                                        **frame_options)
    self.frame_height_info.grid(row=2, column=0)
    # width
    self.frame_width_info = tk.LabelFrame(self.frame_node, width=300, height=300,\
                                        **frame_options)
    self.frame_width_info.grid(row=3, column=0)
    # pos x
    self.frame_posx_info = tk.LabelFrame(self.frame_node, width=300, height=300,\
                                        **frame_options)
    self.frame_posx_info.grid(row=4, column=0)
    # pos y
    self.frame_posy_info = tk.LabelFrame(self.frame_node, width=300, height=300,\
                                        **frame_options)
    self.frame_posy_info.grid(row=5, column=0)
    # layer
    self.frame_layer_info = tk.LabelFrame(self.frame_node, width=300, height=300,\
                                        **frame_options)
    self.frame_layer_info.grid(row=6, column=0)
    # value
    self.frame_value_info = tk.LabelFrame(self.frame_node, width=300, height=300,\
                                        **frame_options)
    self.frame_value_info.grid(row=7, column=0)
    # use
    self.frame_use_info = tk.LabelFrame(self.frame_node, width=300, height=300,\
                                        **frame_options)
    self.frame_use_info.grid(row=8, column=0)
    # useby
    self.frame_useby_info = tk.LabelFrame(self.frame_node, width=300, height=300,\
                                        **frame_options)
    self.frame_useby_info.grid(row=9, column=0)
    # intention
    self.frame_intention_info = tk.LabelFrame(self.frame_node, width=300, height=300,\
                                        **frame_options)
    self.frame_intention_info.grid(row=10, column=0)
    # rationale
    self.frame_rationale_info = tk.LabelFrame(self.frame_node, width=300, height=300,\
                                        **frame_options)
    self.frame_rationale_info.grid(row=11, column=0)
    # annotation
    self.frame_annotation_info = tk.LabelFrame(self.frame_node, width=300, height=300,\
                                        **frame_options)
    self.frame_annotation_info.grid(row=12, column=0)
    # tool
    self.frame_tool_info = tk.LabelFrame(self.frame_node, width=300, height=300,\
                                        **frame_options)
    self.frame_tool_info.grid(row=13, column=0)
    # Export
    self.frame_export = tk.LabelFrame(self.master, text="Export", width=250, height=70,\
                                      **frame_options)
    self.frame_export.place(x=10, y=575)
    # Button
    self.frame_button = tk.LabelFrame(self.master, text="Button", width=250, height=70,\
                                      **frame_options)
    self.frame_button.place(x=490, y=575)
    # use関連付け
    self.frame_use_relation = tk.LabelFrame(self.master, text="USE関連付け", width=250, height=70,\
                                      **frame_options)
    self.frame_use_relation.place(x=708, y=575)
    # Container EP list
    self.frame_con_ep_list = tk.LabelFrame(self.master, text="コンテナとして埋め込むEP List", width=160, height=150,\
                                    **frame_options)
    self.frame_con_ep_list.place(x=1030, y=0)
    # 埋め込む、削除frame
    self.frame_excute = tk.LabelFrame(self.master, text="", width=160, height=150,\
                                    **frame_options)
    self.frame_excute.place(x=1030, y=152)

  #------------------------------------------------------------------
  # ノード情報ウィジェットを作成
  #------------------------------------------------------------------
  def create_node_info_widget(self):
    #------------------------------------------------------------------
    # ノード情報
    #------------------------------------------------------------------
    # id
    self.node_info_id = tk.Label(self.frame_id_info, justify="left", \
      font=("MSゴシック", "8", "bold"), width=20, text="Id:\t                         ", relief="raised")
    self.node_info_id.grid(row=0, column=0)
    self.id_info = tk.Text(self.frame_id_info, width=22, height=1)
    self.id_info.grid(row=0, column=1)
    self.id_info.configure(state="disabled")

    # action
    self.node_info_actionType = tk.Label(self.frame_action_info, justify="left", \
      font=("MSゴシック", "8", "bold"), width=20, text="ActionType:\t         ", relief="raised")
    self.node_info_actionType.grid(row=0, column=0)
    self.action_info = tk.Text(self.frame_action_info, width=22, height=1, wrap = tk.WORD)
    self.action_info.grid(row=0, column=1)
    self.action_info.configure(state="disabled")

    # height
    self.node_info_height = tk.Label(self.frame_height_info, justify="left", \
      font=("MSゴシック", "8", "bold"), width=20, text="GeoBoundingHeight:     ", relief="raised")
    self.node_info_height.grid(row=0, column=0)
    self.height_info = tk.Text(self.frame_height_info, width=22, height=1, wrap = tk.WORD)
    self.height_info.grid(row=0, column=1)
    self.height_info.configure(state="disabled")

    # width
    self.node_info_width = tk.Label(self.frame_width_info, justify="left", \
      font=("MSゴシック", "8", "bold"), width=20, text="GeoBoundingWidth:      ", relief="raised")
    self.node_info_width.grid(row=0, column=0)
    self.width_info = tk.Text(self.frame_width_info, width=22, height=1, wrap = tk.WORD)
    self.width_info.grid(row=0, column=1)
    self.width_info.configure(state="disabled")

    # posx
    self.node_info_posx = tk.Label(self.frame_posx_info, justify="left", \
      font=("MSゴシック", "8", "bold"), width=20, text="GeoBoundingX:\t          ", relief="raised")
    self.node_info_posx.grid(row=0, column=0)
    self.posx_info = tk.Text(self.frame_posx_info, width=22, height=1, wrap = tk.WORD)
    self.posx_info.grid(row=0, column=1)
    self.posx_info.configure(state="disabled")

    # posy
    self.node_info_posy = tk.Label(self.frame_posy_info, justify="left", \
      font=("MSゴシック", "8", "bold"), width=20, text="GeoBoundingY:              ", relief="raised")
    self.node_info_posy.grid(row=0, column=0)
    self.posy_info = tk.Text(self.frame_posy_info, width=22, height=1, wrap = tk.WORD)
    self.posy_info.grid(row=0, column=1)
    self.posy_info.configure(state="disabled")

    # layer
    self.node_info_layer = tk.Label(self.frame_layer_info, justify="right", \
      font=("MSゴシック", "8", "bold"), width=20, text="Layer:        \t          ", relief="raised")
    self.node_info_layer.grid(row=0, column=0)
    self.layer_info = tk.Text(self.frame_layer_info, width=22, height=1, wrap = tk.WORD)
    self.layer_info.grid(row=0, column=1)
    self.layer_info.configure(state="disabled")

    # value
    self.node_info_value = tk.Label(self.frame_value_info, justify="right", \
      font=("MSゴシック", "8", "bold"), width=20, height=3, text="Value:        \t          ", relief="raised")
    self.node_info_value.grid(row=0, column=0)
    self.value_info = scrolledtext.ScrolledText(self.frame_value_info, width=20, height=1, wrap = tk.WORD)
    self.value_info.grid(row=0, column=1)
    self.value_info.configure(state="disabled")

    # use
    self.node_info_use = tk.Label(self.frame_use_info, justify="right", \
      font=("MSゴシック", "8", "bold"), width=20, height=3, text="Use:        \t          ", relief="raised")
    self.node_info_use.grid(row=0, column=0)
    self.use_info = scrolledtext.ScrolledText(self.frame_use_info, width=20, height=1, wrap = tk.WORD)
    self.use_info.grid(row=0, column=1)
    self.use_info.configure(state="disabled")

    # use
    self.node_info_useby = tk.Label(self.frame_useby_info, justify="right", \
      font=("MSゴシック", "8", "bold"), width=20, height=3, text="UseBy:        \t          ", relief="raised")
    self.node_info_useby.grid(row=0, column=0)
    self.useby_info = scrolledtext.ScrolledText(self.frame_useby_info, width=20, height=1, wrap = tk.WORD)
    self.useby_info.grid(row=0, column=1)
    self.useby_info.configure(state="disabled")

    #------------------------------------------------------------------
    # 補足情報
    #------------------------------------------------------------------
    # Intention
    self.node_info_intention = tk.Label(self.frame_intention_info, justify="right", \
      font=("MSゴシック", "8", "bold"), width=20, height=3, text="Intention:        \t          ", relief="raised")
    self.node_info_intention.grid(row=0, column=0)
    self.intention_info = scrolledtext.ScrolledText(self.frame_intention_info, width=20, height=1, wrap = tk.WORD)
    self.intention_info.grid(row=0, column=1)
    self.intention_info.configure(state="disabled")

    # Rationale
    self.node_info_rationale = tk.Label(self.frame_rationale_info, justify="right", \
      font=("MSゴシック", "8", "bold"), width=20,  height=3, text="Rationale:        \t          ", relief="raised")
    self.node_info_rationale.grid(row=0, column=0)
    self.rationale_info = scrolledtext.ScrolledText(self.frame_rationale_info, width=20, height=1, wrap = tk.WORD)
    self.rationale_info.grid(row=0, column=1)
    self.rationale_info.configure(state="disabled")

    # Annotation
    self.node_info_annotation = tk.Label(self.frame_annotation_info, justify="right", \
      font=("MSゴシック", "8", "bold"), width=20,  height=3, text="Annotation:        \t          ", relief="raised")
    self.node_info_annotation.grid(row=0, column=0)
    self.annotation_info = scrolledtext.ScrolledText(self.frame_annotation_info, width=20, height=1, wrap = tk.WORD)
    self.annotation_info.grid(row=0, column=1)
    self.annotation_info.configure(state="disabled")

    # Tool
    self.node_info_tool = tk.Label(self.frame_tool_info, justify="right", \
      font=("MSゴシック", "8", "bold"), width=20,  height=3, text="Tool:        \t          ", relief="raised")
    self.node_info_tool.grid(row=0, column=0)
    self.tool_info = scrolledtext.ScrolledText(self.frame_tool_info, width=20, height=1, wrap = tk.WORD)
    self.tool_info.grid(row=0, column=1)
    self.tool_info.configure(state="disabled")

  #------------------------------------------------------------------
  # 階層ノード情報設定
  #------------------------------------------------------------------
  def get_hierarchy_node_list(self, GPM_uri):

    # 元のtreeviewを削除
    if(self.treeview.get_children):
      for item in self.treeview.get_children():
        self.treeview.delete(item)

    #グラフにあるコンテナ数を取得
    container_count = get_graph_container_count(self.selected_graph_name)

    (self.node_list, self.node_id_list, self.treeview_item_list) = get_top_node_list(self.selected_graph_name)

    # Top node listに選択したuri以外のnodeを削除
    top_node_list = copy.copy(self.node_list["top"])
    top_node_id_list = copy.copy(self.node_id_list["top"])

    for index, top_node in enumerate(top_node_list):
      if(not top_node in self.top_node_list_uri[GPM_uri]):
        self.node_list["top"].remove(top_node)
        delete_node_id = top_node_id_list[index]
        self.node_id_list["top"].remove(delete_node_id)

    for node in self.node_list["top"]:
      parent = self.treeview.insert("", tk.END, text=node, open=False)
      self.treeview_item_list["top"].append(parent)
      self.get_member_node_list(node, container_count, parent)

  #------------------------------------------------------------------
  # member情報を取得
  #------------------------------------------------------------------
  def get_member_node_list(self, top_node, count, parent_node):

    if(count == 0):
      return

    top_node = top_node.replace("<br>","")
    query_top_node = '"' + top_node + '"'
    query = """
    PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
    SELECT ?s
    WHERE {
          graph """ + self.selected_graph_name + """ {
            ?s ?p pd3:Container;
            pd3:value ?value.
        FILTER(?value = """ + query_top_node + """ )
        }
    }
    """
    query_result = get_fuseki_data_json(query)
    if(len(query_result) > 0):
      self.node_list[top_node] = []
      self.node_id_list[top_node] = []
      self.treeview_item_list[top_node]= []
      count -= 1
      for result in query_result:
        container = result["s"]["value"]

      query_2 = """
      PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
      SELECT ?id ?value
      WHERE {
            graph """ + self.selected_graph_name + """ {
              ?s ?p pd3:Action;
              pd3:attribution ?parent;
              pd3:id ?id;
              pd3:value ?value.
          FILTER(?parent = <""" + container + """> )
          }
      }
      """
      query_result = get_fuseki_data_json(query_2)
      for result in query_result:
        value = result["value"]["value"]
        id = result["id"]["value"]
        self.node_list[top_node].append(value)
        self.node_id_list[top_node].append(id)
        #return self.get_member_node_list(value, count)

      for node in self.node_list[top_node]:
        child = self.treeview.insert(parent_node, tk.END, text=node, open=False)
        self.treeview_item_list[top_node].append(child)
        self.get_member_node_list(node, count, child)
    else:
      return

  #------------------------------------------------------------------
  # top level の uri listを取得
  #------------------------------------------------------------------
  def get_uri_list(self, node_list):

    self.top_node_list_uri = {}
    self.con_ep_list = []
    for node in node_list['top']:
      index = 0
      # nodeのuriを取得
      node_str = '"' + node + '"'
      query = """
          PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
          SELECT ?s ?id
          WHERE {
                graph """ + self.selected_graph_name + """ {
                  ?s ?p pd3:Action;
                  pd3:value ?value;
                  pd3:id ?id
                  FILTER(?value = """ + node_str + """)
              }
          }
          """
      query_result = get_fuseki_data_json(query)
      for result in query_result:
        temp_subject = result["s"]["value"]
        id = result["id"]["value"]
        node_subject = temp_subject.replace(id, "")
        if(node_subject not in self.top_node_list_uri):
          self.top_node_list_uri[node_subject]=[]
          self.listbox_gpm_uri_list.insert(index, node_subject)
          # container ep listに追加
          self.con_ep_list.append(node_subject)
          index += 1
        self.top_node_list_uri[node_subject].append(node)

  #------------------------------------------------------------------
  # グラフを選択処理
  #------------------------------------------------------------------
  def select_graph(self, event):
    w = event.widget
    select_index = int(w.curselection()[0])
    self.selected_graph = self.graph_list[select_index]

    self.selected_graph_name = "<" + self.selected_graph + ">"

    # treeviewを更新のため、元のtreeviewを削除
    if(self.treeview.get_children):
      for item in self.treeview.get_children():
        self.treeview.delete(item)

    # gpm listの選択をクリア
    self.listbox_gpm_uri_list.delete(0, tk.END)

    # ノード情報をクリア
    self.clear_node_info()

    # container ep listをクリア
    self.list_con_ep.delete(0, tk.END)

    (self.node_list, self.node_id_list, self.treeview_item_list) = get_top_node_list(self.selected_graph_name)

    self.get_uri_list(self.node_list)

    # グラフを表示のため、ttlファイルを取得
    query_ttl_file = """
        PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
        CONSTRUCT { ?s ?p ?o}
        WHERE {
              graph """ + self.selected_graph_name + """ {
                    ?s ?p ?o;
            }
        }
    """
    get_ttlfile_data(query_ttl_file, self.ttl_file)

  #------------------------------------------------------------------
  # treeviewを選択処理
  #------------------------------------------------------------------
  def tree_select(self, event):
    # node名を取得
    record_id = self.treeview.focus()
    current_item = self.treeview.item(record_id)["text"]

    # node idを取得
    for key in self.treeview_item_list.keys():
      for index, item in enumerate(self.treeview_item_list[key]):
        if(item == record_id):
          node_id = self.node_id_list[key][index]
          new_graph_level = key

    # ノード情報表示xをクリア
    self.clear_node_info()

    # ノード情報を取得(id, actionType, layer, position など)
    (id, action, height, width, pos_x, pos_y, layer) = get_node_info(self.selected_graph_name, node_id)

    (use, useby) = get_use_info(self.selected_graph_name, node_id)

    # ノード情報を表示
    self.node_info_update(self.id_info, id)
    self.node_info_update(self.action_info, action)
    self.node_info_update(self.height_info, height)
    self.node_info_update(self.width_info, width)
    self.node_info_update(self.posx_info, pos_x)
    self.node_info_update(self.posy_info, pos_y)
    self.node_info_update(self.layer_info, layer)
    self.node_info_update(self.value_info, current_item)

    self.node_info_update(self.use_info, use)
    self.node_info_update(self.useby_info, ', '.join(useby))

    # クエリを投げるため、ノードの主語を取得
    target_node = get_supplement_info(self.selected_graph_name, node_id)
    # 補足情報を取得と表示(intention, annotation, rationale, objectなど)
    self.set_supplement_info(target_node)

    image_path = "selected_graph.png"
    # 選択したノードの層と表示しているグラフの層が異なる場合、グラフ更新
    if(new_graph_level != self.graph_level):
      self.graph_level = new_graph_level

      (self.graph_node_list, self.graph_edge_list, self.graph_label_list, self.graph_position) = \
        get_graph_data_by_ttlfile(self.node_id_list[self.graph_level], self.ttl_file)
    
    # 画像を設定
    set_image(image_path, self.graph_node_list, self.graph_edge_list, self.graph_label_list, [node_id],\
              self.graph_position)

    # 画像を更新
    selected_image=Image.open(image_path)
    self.change_image(selected_image)

  #------------------------------------------------------------------
  # ノード情報表示をクリア処理
  #------------------------------------------------------------------
  def clear_node_info(self):
    self.node_info_clear(self.id_info)
    self.node_info_clear(self.action_info)
    self.node_info_clear(self.height_info)
    self.node_info_clear(self.width_info)
    self.node_info_clear(self.posx_info)
    self.node_info_clear(self.posy_info)
    self.node_info_clear(self.layer_info)
    self.node_info_clear(self.value_info)
    self.node_info_clear(self.use_info)
    self.node_info_clear(self.useby_info)

    self.node_info_clear(self.intention_info)
    self.node_info_clear(self.rationale_info)
    self.node_info_clear(self.annotation_info)
    self.node_info_clear(self.tool_info)

  #------------------------------------------------------------------
  # ノード補足情報を取得と表示処理
  #------------------------------------------------------------------
  def set_supplement_info(self, node):
    #------------------------------------------------------------------
    # intentionを取得
    #------------------------------------------------------------------
    (intention_id, intention_value) = get_supplement_detail_info(self.selected_graph_name, "intention", node)

    if(intention_id is not None) and (intention_value is not None):
      self.node_info_update(self.intention_info, intention_value)
    
    #------------------------------------------------------------------
    # rationaleを取得
    #------------------------------------------------------------------
    (rationale_id, rationale_value) = get_supplement_detail_info(self.selected_graph_name, "rationale", node)

    if(rationale_id is not None) and (rationale_value is not None):
      self.node_info_update(self.rationale_info, rationale_value)

    #------------------------------------------------------------------
    # annotationを取得
    #------------------------------------------------------------------
    (annotation_id, annotation_value) = get_supplement_detail_info(self.selected_graph_name, "annotation", node)

    if(annotation_id is not None) and (annotation_value is not None):
      self.node_info_update(self.annotation_info, annotation_value)

    #------------------------------------------------------------------
    # toolを取得
    #------------------------------------------------------------------
    (tool_id, tool_value) = get_supplement_detail_info(self.selected_graph_name, "tool/knowledge", node)

    if(tool_value is not None):
      self.node_info_update(self.tool_info, tool_value)

  #--------------------------------------------
  # グラフ画像の入り替え処理
  #--------------------------------------------
  def change_image(self, img):
    #canvasの書き換え
    self.graph_canvas.photo = ImageTk.PhotoImage(img)
    self.graph_canvas.itemconfig(self.image_on_canvas, image=self.graph_canvas.photo)

  #------------------------------------------------------------------
  # グラフのTOP部分を表示
  #------------------------------------------------------------------
  def show_top_graph(self):
    # グラフ情報を取得
    (self.graph_node_list, self.graph_edge_list, self.graph_label_list, self.graph_position) = \
      get_graph_data_by_ttlfile(self.node_id_list["top"], self.ttl_file)

    # 最上層グラフを表示
    image_path = "top_graph.png"
    self.graph_level = "top"
    set_image(image_path, self.graph_node_list, self.graph_edge_list, self.graph_label_list, [],\
            self.graph_position)
    top_image=Image.open(image_path)

    self.graph_canvas.photo = ImageTk.PhotoImage(top_image)
    self.image_on_canvas = self.graph_canvas.create_image(0, 0, anchor='nw', image=self.graph_canvas.photo)

  #------------------------------------------------------------------
  # treeviewとグラフで確認したいGPMを設定
  #------------------------------------------------------------------
  def set_gpm(self):
    selected_GPM_index = self.listbox_gpm_uri_list.curselection()
    if(len(selected_GPM_index)==0):
      messagebox.showerror('Error', 'No data selected!')
    else:
      self.gpm_uri= self.listbox_gpm_uri_list.get(selected_GPM_index)

    #self.listbox_gpm_uri_list.select_clear(0, tk.END)
    #self.listbox_gpm_uri_list.delete(selected_GPM_index)

    # treeviewを表示
    self.get_hierarchy_node_list(self.gpm_uri)

    # グラフを表示
    self.show_top_graph()

    # container埋め込むリストにEP listを表示
    self.get_container_ep_list(self.gpm_uri)

  #------------------------------------------------------------------
  # コンテナとして埋め込みEPを取得と設定
  #------------------------------------------------------------------
  def get_container_ep_list(self, selected_uri):
      for index, item in enumerate(self.con_ep_list):
        if(item != selected_uri):
          self.list_con_ep.insert(index, item)

  #------------------------------------------------------------------
  # コンテナとして埋め込み処理
  #------------------------------------------------------------------
  def insert_member_event(self):

    # treeviewで選択したノード名とidを取得
    (current_item, current_node_id) = self.get_selected_node_id()

    # treeviewで選択したノードのuriを取得
    query = """
        PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
        SELECT ?s ?id
        WHERE {
              graph """ + self.selected_graph_name + """ {
                ?s pd3:id ?id
            FILTER(?id = """ + '"' + current_node_id + '"' + """)
            }
        }
    """
    query_result = get_fuseki_data_json(query)
    for result in query_result:
      subject = result["s"]["value"]
      id = result["id"]["value"]
      node_uri = subject.replace(id, "")

    # member uriを取得
    selected_member_index = self.list_con_ep.curselection()
    if(len(selected_member_index)==0):
      messagebox.showerror('Error', 'No uri data selected!')
    else:
      member_uri= self.list_con_ep.get(selected_member_index)

    # membertを追加
    insert_member(self.selected_graph_name, current_item, current_node_id, node_uri, member_uri)

    # treeview 更新
    self.get_hierarchy_node_list(self.gpm_uri)

    # graph 更新
    self.show_top_graph()

  #------------------------------------------------------------------
  # コンテナメンバー削除処理
  #------------------------------------------------------------------
  def delete_member_event(self):

    # treeviewで選択したノードnode名とidを取得
    (current_item, current_node_id) = self.get_selected_node_id()

    if not current_item in self.node_list.keys():
      messagebox.showerror('Error', '選択したノードがメンバーはありません!')

    # メンバーリストを取得
    member_id_list = self.node_id_list[current_item]
    member_list = self.node_list[current_item]

    # treeviewで選択したノードのuriを取得
    query = """
        PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
        SELECT ?s ?id
        WHERE {
              graph """ + self.selected_graph_name + """ {
                ?s pd3:id ?id
            FILTER(?id = """ + '"' + current_node_id + '"' + """)
            }
        }
    """
    query_result = get_fuseki_data_json(query)
    for result in query_result:
      subject = result["s"]["value"]
      id = result["id"]["value"]
      node_uri = subject.replace(id, "")

    delete_member(self.selected_graph_name, node_uri, \
                  current_item, current_node_id, member_list, member_id_list)

    # ep list更新


    # treeview 更新
    self.get_hierarchy_node_list(self.gpm_uri)

    # graph 更新
    self.show_top_graph()

  #------------------------------------------------------------------
  # use関連付け追加処理
  #------------------------------------------------------------------
  def insert_use_event(self):

    # 選択したノード名とidを取得
    (current_item, current_node_id) = self.get_selected_node_id()

    # 選択したノードのuriを取得
    query = """
        PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
        SELECT ?s ?id
        WHERE {
              graph """ + self.selected_graph_name + """ {
                ?s pd3:id ?id
            FILTER(?id = """ + '"' + current_node_id + '"' + """)
            }
        }
    """
    query_result = get_fuseki_data_json(query)
    for result in query_result:
      subject = result["s"]["value"]
      id = result["id"]["value"]
      node_uri = subject.replace(id, "")

    log_graph = self.log_graph.get()
    log_graph_uri = self.log_uri.get()
    log_graph_node = self.log_node.get()

    insert_use(self.selected_graph_name, node_uri, current_node_id, log_graph, log_graph_uri, log_graph_node)

    # treeview 更新
    self.get_hierarchy_node_list(self.gpm_uri)

    # graph 更新
    self.show_top_graph()

  #------------------------------------------------------------------
  # use関連付け削除処理
  #------------------------------------------------------------------
  def delete_use_event(self):

    # 選択したノード名とidを取得
    (current_item, current_node_id) = self.get_selected_node_id()

    # 選択したノードのuriを取得
    query = """
        PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
        SELECT ?s ?id
        WHERE {
              graph """ + self.selected_graph_name + """ {
                ?s pd3:id ?id
            FILTER(?id = """ + '"' + current_node_id + '"' + """)
            }
        }
    """
    query_result = get_fuseki_data_json(query)
    for result in query_result:
      subject = result["s"]["value"]
      id = result["id"]["value"]
      node_uri = subject.replace(id, "")

    log_graph = self.log_graph.get()
    log_graph_uri = self.log_uri.get()
    log_graph_node = self.log_node.get()

    delete_use(self.selected_graph_name, node_uri, current_node_id, log_graph, log_graph_uri, log_graph_node)

  #--------------------------------------------
  # ノード情報更新処理
  #--------------------------------------------
  def node_info_update(self, widget, data):
    widget.configure(state="normal")
    widget.insert(1.0, data)
    widget.configure(state="disabled")

  #------------------------------------------------------------------
  # ノード情報クリア処理
  #------------------------------------------------------------------
  def node_info_clear(self, widget):
    widget.configure(state="normal")
    widget.delete("1.0","end")
    widget.configure(state="disabled")

  #------------------------------------------------------------------
  # treeviewで選択したノードのidを取得
  #------------------------------------------------------------------
  def get_selected_node_id(self):

    # ノードnode名を取得
    record_id = self.treeview.focus()
    if(not record_id):
      messagebox.showerror('Error', 'No node selected!')
    else:
      current_item = self.treeview.item(record_id)["text"]

    # ノードidを取得
    for key in self.treeview_item_list.keys():
      for index, item in enumerate(self.treeview_item_list[key]):
        if(record_id == item):
          current_node_id = self.node_id_list[key][index]

    return current_item, current_node_id

  #------------------------------------------------------------------
  # ログウインドウを開く
  #------------------------------------------------------------------
  def open_log_window(self):
    logWindow(self.log_graph, self.log_uri, self.log_node, self)


In [3]:
class logWindow(tk.Toplevel):
  
  def __init__(self, log_graph, log_uri, log_node, master = None):
    super().__init__(master=master)
    # 主ウインドウを渡す変数
    self.log_graph = log_graph
    self.log_uri = log_uri
    self.log_node = log_node
    self.title("Log Window")
    self.geometry("1200x700")
    self.ttl_file_log = 'ttl_file_log.ttl'
    self.create_widgets()

  #------------------------------------------------------------------
  # ウィジェットを作成
  #------------------------------------------------------------------
  def create_widgets(self):
    #------------------------------------------------------------------
    # LabelFrame
    #------------------------------------------------------------------
    self.create_frame_widget()
    #------------------------------------------------------------------
    # listbox widget (Graph Info)
    #------------------------------------------------------------------
    self.scrollbar_ep_info_x = tk.Scrollbar(self.frame_ep_info_list, orient=tk.HORIZONTAL)
    self.scrollbar_ep_info_x.pack(side="bottom", fill="x")
    self.scrollbar_ep_info_y = tk.Scrollbar(self.frame_ep_info_list, orient=tk.VERTICAL)
    self.scrollbar_ep_info_y.pack(side="right", fill="y")

    self.listbox_ep_value = tk.StringVar()
    self.graph_list = get_graph()
    self.listbox_ep_value.set(self.graph_list)
    self.listbox_ep_info = tk.Listbox(self.frame_ep_info_list, height=6, width=30, selectmode=tk.SINGLE,\
                            listvariable=self.listbox_ep_value, \
                            exportselection=False, xscrollcommand=self.scrollbar_ep_info_x.set,\
                            yscrollcommand=self.scrollbar_ep_info_y.set)  
    self.listbox_ep_info.pack(fill="both")
    self.listbox_ep_info.bind("<<ListboxSelect>>", self.select_graph_log)

    self.scrollbar_ep_info_x.config(command=self.listbox_ep_info.xview)
    self.scrollbar_ep_info_y.config(command=self.listbox_ep_info.yview)
    #------------------------------------------------------------------
    # treeview widget
    #------------------------------------------------------------------
    self.treeview = ttk.Treeview(self.frame_hierarchy_list)
    # 階層列設定
    self.treeview.column("#0", width=180, minwidth=500)
    # 階層列(ツリーカラム)の見出し設定。階層列とする。
    self.treeview.heading("#0")
    self.treeview.bind("<<TreeviewSelect>>", self.log_tree_select)
    self.treeview.grid(row=1, column=1, sticky="nsew")

    tree_scrollbar_y = ttk.Scrollbar(self.frame_hierarchy_list, orient="vertical",\
                      command=self.treeview.yview)
    tree_scrollbar_y.grid(row=1, column=2, sticky="nsew")
    self.treeview.configure(yscrollcommand=tree_scrollbar_y.set)

    tree_scrollbar_x = ttk.Scrollbar(self.frame_hierarchy_list, orient="horizontal",\
                      command=self.treeview.xview)
    tree_scrollbar_x.grid(row=2, column=1, sticky="nsew")
    self.treeview.configure(xscrollcommand=tree_scrollbar_x.set)
    #------------------------------------------------------------------
    # ノード情報ウィジェットを作成
    #------------------------------------------------------------------
    self.create_node_info_widget()
    #------------------------------------------------------------------
    # Button widget
    #------------------------------------------------------------------
    self.button_log_node_select = tk.Button(self.frame_button, text=" 選択 ", width=7, height=2, \
                                command=self.log_node_select)
    self.button_log_node_select.grid(row=1, column=0, padx=5)
    #------------------------------------------------------------------
    # canvas
    #------------------------------------------------------------------
    self.graph_canvas = tk.Canvas(self)
    self.graph_canvas.place(x=0, y=0, width=480, height=600)
    #canvas スクロールバー
    self.canvas_bar_y = tk.Scrollbar(self.graph_canvas, orient=tk.VERTICAL)
    self.canvas_bar_x = tk.Scrollbar(self.graph_canvas, orient=tk.HORIZONTAL)
    self.canvas_bar_y.pack(side=tk.RIGHT, fill=tk.Y)
    self.canvas_bar_x.pack(side=tk.BOTTOM, fill=tk.X)
    self.canvas_bar_y.config(command=self.graph_canvas.yview)
    self.canvas_bar_x.config(command=self.graph_canvas.xview)
    self.graph_canvas.config(yscrollcommand=self.canvas_bar_y.set, xscrollcommand=self.canvas_bar_x.set)
    # Canvasのスクロール範囲を設定
    self.graph_canvas.config(scrollregion=(0, 0, 1000, 1000))

  #------------------------------------------------------------------
  # LabelFrameを作成
  #------------------------------------------------------------------
  def create_frame_widget(self):
    frame_options = {"bd": 2, "relief": "ridge", "foreground": "green"}

    self.frame_ep_info_list = tk.LabelFrame(self, text=" Graph Info", width=300, height=120,\
                                            **frame_options)
    self.frame_ep_info_list.place(x=495,y=0)

    self.frame_hierarchy_list = tk.LabelFrame(self, text="Action List", width=300, height=240,\
                                            **frame_options)
    self.frame_hierarchy_list.place(x=495,y=140)

    self.frame_node = tk.LabelFrame(self, text="Infromation", width=300, height=300,\
                                    **frame_options)
    self.frame_node.place(x=710, y=0)
    # id
    self.frame_id_info = tk.LabelFrame(self.frame_node, width=300, height=300,\
                                        **frame_options)
    self.frame_id_info.grid(row=0, column=0)
    # action
    self.frame_action_info = tk.LabelFrame(self.frame_node, width=300, height=300,\
                                        **frame_options)
    self.frame_action_info.grid(row=1, column=0)
    # height
    self.frame_height_info = tk.LabelFrame(self.frame_node, width=300, height=300,\
                                        **frame_options)
    self.frame_height_info.grid(row=2, column=0)
    # width
    self.frame_width_info = tk.LabelFrame(self.frame_node, width=300, height=300,\
                                        **frame_options)
    self.frame_width_info.grid(row=3, column=0)
    # pos x
    self.frame_posx_info = tk.LabelFrame(self.frame_node, width=300, height=300,\
                                        **frame_options)
    self.frame_posx_info.grid(row=4, column=0)
    # pos y
    self.frame_posy_info = tk.LabelFrame(self.frame_node, width=300, height=300,\
                                        **frame_options)
    self.frame_posy_info.grid(row=5, column=0)
    # layer
    self.frame_layer_info = tk.LabelFrame(self.frame_node, width=300, height=300,\
                                        **frame_options)
    self.frame_layer_info.grid(row=6, column=0)
    # value
    self.frame_value_info = tk.LabelFrame(self.frame_node, width=300, height=300,\
                                        **frame_options)
    self.frame_value_info.grid(row=7, column=0)
    # use
    self.frame_use_info = tk.LabelFrame(self.frame_node, width=300, height=300,\
                                        **frame_options)
    self.frame_use_info.grid(row=8, column=0)
    # useby
    self.frame_useby_info = tk.LabelFrame(self.frame_node, width=300, height=300,\
                                        **frame_options)
    self.frame_useby_info.grid(row=9, column=0)
    # intention
    self.frame_intention_info = tk.LabelFrame(self.frame_node, width=300, height=300,\
                                        **frame_options)
    self.frame_intention_info.grid(row=10, column=0)
    # rationale
    self.frame_rationale_info = tk.LabelFrame(self.frame_node, width=300, height=300,\
                                        **frame_options)
    self.frame_rationale_info.grid(row=11, column=0)
    # annotation
    self.frame_annotation_info = tk.LabelFrame(self.frame_node, width=300, height=300,\
                                        **frame_options)
    self.frame_annotation_info.grid(row=12, column=0)
    # tool
    self.frame_tool_info = tk.LabelFrame(self.frame_node, width=300, height=300,\
                                        **frame_options)
    self.frame_tool_info.grid(row=13, column=0)
    # obejct
    self.frame_object_info = tk.LabelFrame(self.frame_node, width=300, height=300,\
                                        **frame_options)
    self.frame_object_info.grid(row=14, column=0)

    self.frame_button = tk.LabelFrame(self, text="Button", width=250, height=70,\
                                      **frame_options)
    self.frame_button.place(x=495, y=410)

  #------------------------------------------------------------------
  # ノード情報ウィジェットを作成
  #------------------------------------------------------------------
  def create_node_info_widget(self):
    #------------------------------------------------------------------
    # ノード情報
    #------------------------------------------------------------------
    # id
    self.node_info_id = tk.Label(self.frame_id_info, justify="left", \
      font=("MSゴシック", "8", "bold"), width=20, text="Id:\t                         ", relief="raised")
    self.node_info_id.grid(row=0, column=0)
    self.id_info = tk.Text(self.frame_id_info, width=22, height=1)
    self.id_info.grid(row=0, column=1)
    self.id_info.configure(state="disabled")

    # action
    self.node_info_actionType = tk.Label(self.frame_action_info, justify="left", \
      font=("MSゴシック", "8", "bold"), width=20, text="ActionType:\t         ", relief="raised")
    self.node_info_actionType.grid(row=0, column=0)
    self.action_info = tk.Text(self.frame_action_info, width=22, height=1, wrap = tk.WORD)
    self.action_info.grid(row=0, column=1)
    self.action_info.configure(state="disabled")

    # height
    self.node_info_height = tk.Label(self.frame_height_info, justify="left", \
      font=("MSゴシック", "8", "bold"), width=20, text="GeoBoundingHeight:     ", relief="raised")
    self.node_info_height.grid(row=0, column=0)
    self.height_info = tk.Text(self.frame_height_info, width=22, height=1, wrap = tk.WORD)
    self.height_info.grid(row=0, column=1)
    self.height_info.configure(state="disabled")

    # width
    self.node_info_width = tk.Label(self.frame_width_info, justify="left", \
      font=("MSゴシック", "8", "bold"), width=20, text="GeoBoundingWidth:      ", relief="raised")
    self.node_info_width.grid(row=0, column=0)
    self.width_info = tk.Text(self.frame_width_info, width=22, height=1, wrap = tk.WORD)
    self.width_info.grid(row=0, column=1)
    self.width_info.configure(state="disabled")

    # posx
    self.node_info_posx = tk.Label(self.frame_posx_info, justify="left", \
      font=("MSゴシック", "8", "bold"), width=20, text="GeoBoundingX:\t          ", relief="raised")
    self.node_info_posx.grid(row=0, column=0)
    self.posx_info = tk.Text(self.frame_posx_info, width=22, height=1, wrap = tk.WORD)
    self.posx_info.grid(row=0, column=1)
    self.posx_info.configure(state="disabled")

    # posy
    self.node_info_posy = tk.Label(self.frame_posy_info, justify="left", \
      font=("MSゴシック", "8", "bold"), width=20, text="GeoBoundingY:              ", relief="raised")
    self.node_info_posy.grid(row=0, column=0)
    self.posy_info = tk.Text(self.frame_posy_info, width=22, height=1, wrap = tk.WORD)
    self.posy_info.grid(row=0, column=1)
    self.posy_info.configure(state="disabled")

    # layer
    self.node_info_layer = tk.Label(self.frame_layer_info, justify="right", \
      font=("MSゴシック", "8", "bold"), width=20, text="Layer:        \t          ", relief="raised")
    self.node_info_layer.grid(row=0, column=0)
    self.layer_info = tk.Text(self.frame_layer_info, width=22, height=1, wrap = tk.WORD)
    self.layer_info.grid(row=0, column=1)
    self.layer_info.configure(state="disabled")

    # value
    self.node_info_value = tk.Label(self.frame_value_info, justify="right", \
      font=("MSゴシック", "8", "bold"), width=20, height=3, text="Value:        \t          ", relief="raised")
    self.node_info_value.grid(row=0, column=0)
    self.value_info = scrolledtext.ScrolledText(self.frame_value_info, width=20, height=1, wrap = tk.WORD)
    self.value_info.grid(row=0, column=1)
    self.value_info.configure(state="disabled")

    # use
    self.node_info_use = tk.Label(self.frame_use_info, justify="right", \
      font=("MSゴシック", "8", "bold"), width=20, height=3, text="Use:        \t          ", relief="raised")
    self.node_info_use.grid(row=0, column=0)
    self.use_info = scrolledtext.ScrolledText(self.frame_use_info, width=20, height=1, wrap = tk.WORD)
    self.use_info.grid(row=0, column=1)
    self.use_info.configure(state="disabled")

    # use
    self.node_info_useby = tk.Label(self.frame_useby_info, justify="right", \
      font=("MSゴシック", "8", "bold"), width=20, height=3, text="UseBy:        \t          ", relief="raised")
    self.node_info_useby.grid(row=0, column=0)
    self.useby_info = scrolledtext.ScrolledText(self.frame_useby_info, width=20, height=1, wrap = tk.WORD)
    self.useby_info.grid(row=0, column=1)
    self.useby_info.configure(state="disabled")

    #------------------------------------------------------------------
    # 補足情報
    #------------------------------------------------------------------
    # Intention
    self.node_info_intention = tk.Label(self.frame_intention_info, justify="right", \
      font=("MSゴシック", "8", "bold"), width=20, height=3, text="Intention:        \t          ", relief="raised")
    self.node_info_intention.grid(row=0, column=0)
    self.intention_info = scrolledtext.ScrolledText(self.frame_intention_info, width=20, height=1, wrap = tk.WORD)
    self.intention_info.grid(row=0, column=1)
    self.intention_info.configure(state="disabled")

    # Rationale
    self.node_info_rationale = tk.Label(self.frame_rationale_info, justify="right", \
      font=("MSゴシック", "8", "bold"), width=20,  height=3, text="Rationale:        \t          ", relief="raised")
    self.node_info_rationale.grid(row=0, column=0)
    self.rationale_info = scrolledtext.ScrolledText(self.frame_rationale_info, width=20, height=1, wrap = tk.WORD)
    self.rationale_info.grid(row=0, column=1)
    self.rationale_info.configure(state="disabled")

    # Annotation
    self.node_info_annotation = tk.Label(self.frame_annotation_info, justify="right", \
      font=("MSゴシック", "8", "bold"), width=20,  height=3, text="Annotation:        \t          ", relief="raised")
    self.node_info_annotation.grid(row=0, column=0)
    self.annotation_info = scrolledtext.ScrolledText(self.frame_annotation_info, width=20, height=1, wrap = tk.WORD)
    self.annotation_info.grid(row=0, column=1)
    self.annotation_info.configure(state="disabled")

    # Tool
    self.node_info_tool = tk.Label(self.frame_tool_info, justify="right", \
      font=("MSゴシック", "8", "bold"), width=20,  height=3, text="Tool:        \t          ", relief="raised")
    self.node_info_tool.grid(row=0, column=0)
    self.tool_info = scrolledtext.ScrolledText(self.frame_tool_info, width=20, height=1, wrap = tk.WORD)
    self.tool_info.grid(row=0, column=1)
    self.tool_info.configure(state="disabled")

    # Object
    self.node_info_object = tk.Label(self.frame_object_info, justify="right", \
      font=("MSゴシック", "8", "bold"), width=20,  height=3, text="Object:        \t          ", relief="raised")
    self.node_info_object.grid(row=0, column=0)
    self.object_info = scrolledtext.ScrolledText(self.frame_object_info, width=20, height=1, wrap = tk.WORD)
    self.object_info.grid(row=0, column=1)
    self.object_info.configure(state="disabled")

  #------------------------------------------------------------------
  # グラフを選択処理
  #------------------------------------------------------------------
  def select_graph_log(self, event):
    w = event.widget
    select_index = int(w.curselection()[0])
    self.selected_graph = self.graph_list[select_index]

    self.selected_graph_name = "<" + self.selected_graph + ">"

    # treeviewを更新のため、元のtreeviewを削除
    if(self.treeview.get_children):
      for item in self.treeview.get_children():
        self.treeview.delete(item)

    # gpm listの選択をクリア
    #self.listbox_gpm_uri_list.delete(0, tk.END)

    # ノード情報をクリア
    self.clear_node_info()

    (self.node_list, self.node_id_list, self.treeview_item_list) = get_top_node_list(self.selected_graph_name)

    # treeviewを表示
    self.get_hierarchy_node_list()

    # グラフを表示のため、ttlファイルを取得
    query_ttl_file = """
        PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
        CONSTRUCT { ?s ?p ?o}
        WHERE {
              graph """ + self.selected_graph_name + """ {
                    ?s ?p ?o;
            }
        }
    """
    get_ttlfile_data(query_ttl_file, self.ttl_file_log)

    self.show_top_log_graph()

  #------------------------------------------------------------------
  # ノード情報表示をクリア処理
  #------------------------------------------------------------------
  def clear_node_info(self):
    self.node_info_clear(self.id_info)
    self.node_info_clear(self.action_info)
    self.node_info_clear(self.height_info)
    self.node_info_clear(self.width_info)
    self.node_info_clear(self.posx_info)
    self.node_info_clear(self.posy_info)
    self.node_info_clear(self.layer_info)
    self.node_info_clear(self.value_info)
    self.node_info_clear(self.use_info)
    self.node_info_clear(self.useby_info)

    self.node_info_clear(self.intention_info)
    self.node_info_clear(self.rationale_info)
    self.node_info_clear(self.annotation_info)
    self.node_info_clear(self.tool_info)
    self.node_info_clear(self.object_info)

  #------------------------------------------------------------------
  # グラフのTOP部分を表示
  #------------------------------------------------------------------
  def show_top_log_graph(self):
    # グラフ情報を取得
    (self.graph_node_list, self.graph_edge_list, self.graph_label_list, self.graph_position) = \
      get_graph_data_by_ttlfile(self.node_id_list["top"], self.ttl_file_log)

    # 最上層グラフを表示
    image_path = "top_log_graph.png"
    self.graph_level = "top"
    set_image(image_path, self.graph_node_list, self.graph_edge_list, self.graph_label_list, [],\
            self.graph_position)
    top_image=Image.open(image_path)

    self.graph_canvas.photo = ImageTk.PhotoImage(top_image)
    self.image_on_canvas = self.graph_canvas.create_image(0, 0, anchor='nw', image=self.graph_canvas.photo)

  #------------------------------------------------------------------
  # 階層ノード情報設定
  #------------------------------------------------------------------
  def get_hierarchy_node_list(self):
    # 元のtreeviewを削除
    if(self.treeview.get_children):
      for item in self.treeview.get_children():
        self.treeview.delete(item)

    #グラフにあるコンテナ数を取得
    query = """
    PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
    SELECT (COUNT(?s) AS ?count)
    WHERE {
          graph """ + self.selected_graph_name + """ {
            ?s ?p pd3:Container;
            pd3:value ?value.
        }
    }
    """
    query_result = get_fuseki_data_json(query)
    for result in query_result:
      container_count = int(result["count"]["value"])

    (self.node_list, self.node_id_list, self.treeview_item_list) = get_top_node_list(self.selected_graph_name)

    # Top node listに選択したuri以外のnodeを削除
    """
    top_node_list = copy.copy(self.node_list["top"])
    top_node_id_list = copy.copy(self.node_id_list["top"])

    for index, top_node in enumerate(top_node_list):
      if(not top_node in self.top_node_list_uri[GPM_uri]):
        self.node_list["top"].remove(top_node)
        delete_node_id = top_node_id_list[index]
        self.node_id_list["top"].remove(delete_node_id)
    """
    for node in self.node_list["top"]:
      parent = self.treeview.insert("", tk.END, text=node, open=False)
      self.get_member_node_list(node, container_count, parent)

  #------------------------------------------------------------------
  # member情報を取得
  #------------------------------------------------------------------
  def get_member_node_list(self, top_node, count, parent_node):
    if(count == 0):
      return

    query_top_node = '"' + top_node + '"'
    query = """
    PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
    SELECT ?s
    WHERE {
          graph """ + self.selected_graph_name + """ {
            ?s ?p pd3:Container;
            pd3:value ?value.
        FILTER(?value = """ + query_top_node + """ )
        }
    }
    """
    query_result = get_fuseki_data_json(query)
    if(len(query_result) > 0):
      self.node_list[top_node] = []
      self.node_id_list[top_node] = []
      count -= 1
      for result in query_result:
        container = result["s"]["value"]

      query_2 = """
      PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
      SELECT ?id ?value
      WHERE {
            graph """ + self.selected_graph_name + """ {
              ?s ?p pd3:Action;
              pd3:attribution ?parent;
              pd3:id ?id;
              pd3:value ?value.
          FILTER(?parent = <""" + container + """> )
          }
      }
      """
      query_result = get_fuseki_data_json(query_2)
      for result in query_result:
        value = result["value"]["value"]
        id = result["id"]["value"]
        self.node_list[top_node].append(value)
        self.node_id_list[top_node].append(id)
        #return self.get_member_node_list(value, count)

      for node in self.node_list[top_node]:
        child = self.treeview.insert(parent_node, tk.END, text=node, open=False)
        self.get_member_node_list(node, count, child)
    else:
      return

  #------------------------------------------------------------------
  # treeviewを選択処理
  #------------------------------------------------------------------
  def log_tree_select(self, event):
    # node名を取得
    record_id = self.treeview.focus()
    current_item = self.treeview.item(record_id)["text"]

    # node idを取得
    for key in self.node_list.keys():
      for index, node in enumerate(self.node_list[key]):
        if(node == current_item):
          node_id = self.node_id_list[key][index]
          new_graph_level = key

    # ノード情報表示をクリア
    self.clear_node_info()

    # ノード情報を取得(id, actionType, layer, position など)
    (id, action, height, width, pos_x, pos_y, layer) = get_node_info(self.selected_graph_name, node_id)

    (use, useby) = get_use_info(self.selected_graph_name, node_id)

    # ノード情報を表示
    self.node_info_update(self.id_info, id)
    self.node_info_update(self.action_info, action)
    self.node_info_update(self.height_info, height)
    self.node_info_update(self.width_info, width)
    self.node_info_update(self.posx_info, pos_x)
    self.node_info_update(self.posy_info, pos_y)
    self.node_info_update(self.layer_info, layer)
    self.node_info_update(self.value_info, current_item)

    self.node_info_update(self.use_info, use)
    self.node_info_update(self.useby_info, useby)

    # クエリを投げるため、ノードの主語を取得
    target_node = get_supplement_info(self.selected_graph_name, node_id)
    # 補足情報を取得と表示(intention, annotation, rationale, objectなど)
    self.set_supplement_info(target_node)

    image_path = "selected_graph.png"
    # 選択したノードの層と表示しているグラフの層が異なる場合、グラフ更新
    if(new_graph_level != self.graph_level):
      self.graph_level = new_graph_level

      (self.graph_node_list, self.graph_edge_list, self.graph_label_list, self.graph_position) = \
        get_graph_data_by_ttlfile(self.node_id_list[self.graph_level], self.ttl_file_log)
    
    # 画像を設定
    set_image(image_path, self.graph_node_list, self.graph_edge_list, self.graph_label_list, [node_id],\
              self.graph_position)

    # 画像を更新
    selected_image=Image.open(image_path)
    self.change_image(selected_image)

  #------------------------------------------------------------------
  # ノード補足情報を取得と表示処理
  #------------------------------------------------------------------
  def set_supplement_info(self, node):

    # intentionを取得
    (intention_id, intention_value) = get_supplement_detail_info(self.selected_graph_name, "intention", node)

    if(intention_id is not None) and (intention_value is not None):
      self.node_info_update(self.intention_info, intention_value)
    
    # rationaleを取得
    (rationale_id, rationale_value) = get_supplement_detail_info(self.selected_graph_name, "rationale", node)

    if(rationale_id is not None) and (rationale_value is not None):
      self.node_info_update(self.rationale_info, rationale_value)

    # annotationを取得
    (annotation_id, annotation_value) = get_supplement_detail_info(self.selected_graph_name, "annotation", node)

    if(annotation_id is not None) and (annotation_value is not None):
      self.node_info_update(self.annotation_info, annotation_value)

    # toolを取得
    (tool_id, tool_value) = get_supplement_detail_info(self.selected_graph_name, "tool/knowledge", node)

    if(tool_value is not None):
      self.node_info_update(self.tool_info, tool_value)

    # objectを取得
    (object_id, object_value) = get_supplement_detail_info(self.selected_graph_name, "", node)

    if(object_id is not None) and (object_value is not None):
      self.node_info_update(self.object_info, object_value)

  #--------------------------------------------
  # グラフ画像の入り替え処理
  #--------------------------------------------
  def change_image(self, img):
    #canvasの書き換え
    self.graph_canvas.photo = ImageTk.PhotoImage(img)
    self.graph_canvas.itemconfig(self.image_on_canvas, image=self.graph_canvas.photo)

  #--------------------------------------------
  # ログノード選択処理
  #--------------------------------------------
  def log_node_select(self):

    # node名を取得
    record_id = self.treeview.focus()
    current_item = self.treeview.item(record_id)["text"]

    # node idを取得
    for key in self.node_list.keys():
      for index, node in enumerate(self.node_list[key]):
        if(node == current_item):
          node_id = self.node_id_list[key][index]
          new_graph_level = key

    # 選択したノードのuriを取得
    query = """
        PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
        SELECT ?s ?id
        WHERE {
              graph """ + self.selected_graph_name + """ {
                ?s pd3:id ?id
            FILTER(?id = """ + '"' + node_id + '"' + """)
            }
        }
    """
    query_result = get_fuseki_data_json(query)
    for result in query_result:
      subject = result["s"]["value"]
      id = result["id"]["value"]
      node_uri = subject.replace(id, "")

    #print(current_item)
    #print(node_id)
    # 主ウインドウに設定する
    self.log_graph.set(self.selected_graph_name)
    self.log_uri.set(node_uri)
    self.log_node.set(node_id)

  #--------------------------------------------
  # ノード情報更新処理
  #--------------------------------------------
  def node_info_update(self, widget, data):
    widget.configure(state="normal")
    widget.insert(1.0, data)
    widget.configure(state="disabled")

  #------------------------------------------------------------------
  # ノード情報クリア処理
  #------------------------------------------------------------------
  def node_info_clear(self, widget):
    widget.configure(state="normal")
    widget.delete("1.0","end")
    widget.configure(state="disabled")

In [4]:
#------------------------------------------------------------------
# グラフにあるコンテナ数を取得
#------------------------------------------------------------------
def get_graph_container_count(graph_name):
  query = """
    PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
    SELECT (COUNT(?s) AS ?count)
    WHERE {
          graph """ + graph_name + """ {
            ?s ?p pd3:Container;
            pd3:value ?value.
        }
  }
  """
  query_result = get_fuseki_data_json(query)
  for result in query_result:
    count = int(result["count"]["value"])

  return count


In [5]:
#------------------------------------------------------------------
# EPにあるコンテナ数を取得
#------------------------------------------------------------------
def get_ep_container_count(graph_name, uri):
  query = """
    PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
    SELECT (COUNT(?s) AS ?count)
    WHERE {
          graph """ + graph_name + """ {
            ?s ?p pd3:Container;
            pd3:value ?value.
          filter(regex(str(?s),""" + '"' + uri + '"' + """)) 
        }
  }
  """
  query_result = get_fuseki_data_json(query)
  for result in query_result:
    count = int(result["count"]["value"])

  return count

In [6]:
#------------------------------------------------------------------
# Fusekiからgraphを取得
#------------------------------------------------------------------
def get_graph():
  query= """PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
        select distinct ?graph
        where {
                graph ?graph {
                  ?s ?p ?o;
                }
        }"""
  graph_list = []

  results = get_fuseki_data_json(query)
  for result in results:
    graph = result["graph"]["value"]
    graph_list.append(graph)

  return(graph_list)

In [7]:
#------------------------------------------------------------------
# 階層Topノードを取得
#------------------------------------------------------------------
def get_top_node_list(selected_graph_name):

  #最上層アクションを取得
  query = """
        PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
        SELECT ?id ?value
        WHERE {
              graph """ + selected_graph_name + """ {
                ?s ?p pd3:Action;
                  pd3:id ?id;
                  pd3:value ?value.
                FILTER NOT EXISTS {?s pd3:attribution ?parent}
            }
        }
  """
  node_list = {}
  node_list["top"] = []
  node_id_list = {}
  node_id_list["top"] = []
  treeview_item_list = {}
  treeview_item_list["top"] = []

  query_result = get_fuseki_data_json(query)
  for result in query_result:
    value = result["value"]["value"]
    id = result["id"]["value"]
    node_list["top"].append(value)
    node_id_list["top"].append(id)
  return node_list, node_id_list, treeview_item_list

In [8]:
# ------------------------------------------------------------------
# ttlファイルを取得
# ------------------------------------------------------------------
def get_ttlfile_data(query, file_path):
    sparql = SPARQLWrapper("http://localhost:3030/akiyama/sparql")
    sparql.setQuery(query)

    sparql.setReturnFormat(TURTLE)
    results = sparql.query().convert()

    with open(file_path, 'w', encoding='utf-8') as f:
        # bytes型をdecodeでstr型に変換する
        f.write(results.decode())


In [9]:
#------------------------------------------------------------------
# グラフデータを取得
#------------------------------------------------------------------
def get_graph_data_by_ttlfile(node_id_list, file_path):

  node_list = []
  edge_list = []
  label_list = {}
  position = {}
  pd3 = Namespace('http://DigitalTriplet.net/2021/08/ontology#')

  with open(file_path, 'r', encoding='utf-8') as f:
    g = Graph()
    g.parse(f)

  for s, p, o in g.triples((None, RDF.type, None)):
    # Action
    if(o == pd3.Action):
      for s1, p1, o1 in g.triples((s, pd3.id, None)):
        id = str(o1)
        if(id in node_id_list):
          node_list.append(id)
          for s2, p2, o2 in g.triples((s, pd3.geoBoundingX, None)):
            postion_x = float(o2)
          for s3, p3, o3 in g.triples((s, pd3.geoBoundingY, None)):
            postion_y = float(o3)
          position[id] = [postion_x, 1000-postion_y]
          for s4, p4, o4 in g.triples((s, pd3.value, None)):
            value = str(o4)
            label_list[id] = value
    # Flow
    elif(o == pd3.Flow):
      for s2, p2, o2 in g.triples((s, pd3.source, None)):
        temp_source = o2
      for s3, p3, o3 in g.triples((temp_source, pd3.id, None)):
        source = str(o3)
      for s2, p2, o2 in g.triples((s, pd3.target, None)):
        temp_target = o2
        for s3, p3, o3 in g.triples((temp_target, pd3.id, None)):
          target = str(o3)
      if(source in node_id_list) and (target in node_id_list):
        edge_list.append((source, target, {"color" : "k"}))

  return node_list, edge_list, label_list, position

In [10]:
#------------------------------------------------------------------
# ノード情報を取得
#------------------------------------------------------------------
def get_node_info(graph_name, node_id):

  query = """
      PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
      PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
      select distinct ?s ?id ?actionType ?height ?width ?pos_x ?pos_y ?layer ?type
      where {
            graph """ + graph_name + """ {
            ?s pd3:id ?id;
              pd3:actionType ?actionType;
              pd3:geoBoundingHeight ?height;
              pd3:geoBoundingWidth ?width;
              pd3:geoBoundingX ?pos_x;
              pd3:geoBoundingY ?pos_y;
              pd3:layer ?layer;
              rdf:type ?type.
            }
            FILTER(?id = """ + '"' + node_id + '"' + """)
  }"""

  query_result = get_fuseki_data_json(query)

  for result in query_result:
    id = result["id"]["value"] 
    action = result["actionType"]["value"]
    height = result["height"]["value"]
    width = result["width"]["value"]
    pos_x = result["pos_x"]["value"]
    pos_y = result["pos_y"]["value"]
    layer = result["layer"]["value"]

  return id, action, height, width, pos_x, pos_y, layer

In [11]:
#------------------------------------------------------------------
# ノードUse, UseBy情報を取得
#------------------------------------------------------------------
def get_use_info(graph_name, node_id):

  useby = []
  query_use = """
            PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
            PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
            select distinct ?s ?id ?use
            where {
                  graph """ + graph_name + """ {
                  ?s pd3:id ?id;
                    pd3:use ?use.
                  }
                  FILTER(?id = """ + '"' + node_id + '"' + """)
      }"""

  query_use_result = get_fuseki_data_json(query_use)
  if(len(query_use_result) > 0):
    for result in query_use_result:
      use = result["use"]["value"]
  else:
    use = ""

  query_useby = """
            PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
            PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
            select distinct ?s ?id ?useby
            where {
                  graph """ + graph_name + """ {
                  ?s pd3:id ?id;
                    pd3:useBy ?useby.
                  }
                  FILTER(?id = """ + '"' + node_id + '"' + """)
      }"""

  query_useby_result = get_fuseki_data_json(query_useby)
  if(len(query_useby_result) > 0):
    for result in query_useby_result:
      useby.append(result["useby"]["value"])

  return use, useby


In [12]:
#------------------------------------------------------------------
# Fusekiにノードの主語を取得
#------------------------------------------------------------------
def get_supplement_info(graph_name, node_id):

  query = """
      PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
      SELECT ?s
      WHERE {
            graph """ + graph_name + """ {
              ?s pd3:id ?id.
          FILTER(?id = """ + '"' + node_id + '"' + """)
          }
      }
  """
  query_result = get_fuseki_data_json(query)
  for result in query_result:
    target = result["s"]["value"]

  target = "<" + target + ">"

  return target

In [13]:
#------------------------------------------------------------------
# ノード補足情報を取得
#------------------------------------------------------------------
def get_supplement_detail_info(graph_name, type, taget_id):

  # オブジェクトではない場合
  if(type != ""):
    type = '"'+ type + '"'

    query= """
        PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
        SELECT ?s ?id ?value
        WHERE {
              graph """ + graph_name + """ {
                ?s pd3:target ?target;
                pd3:arcType ?arcType;
                pd3:id ?id;
                pd3:value ?value.
            FILTER(?arcType = """ + type + """)
            FILTER(?target = """ + taget_id + """)
            }
        }
    """
    results = get_fuseki_data_json(query)
    if(len(results) > 0):
      for result in results:
        subject = result["s"]["value"]
        id = result["id"]["value"]
        value = result["value"]["value"]

      return id, value

  # オブジェクトの場合
  else:
    query_2 = """
        PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
        SELECT ?s 
        WHERE {
              graph """ + graph_name + """ {
                ?s pd3:target ?target;
                pd3:arcType ?arcType;
                pd3:id ?id;
                pd3:value ?value.
            FILTER(?arcType = "tool/knowledge")
            FILTER(?target = """ + taget_id + """)
            }
        }
    """
    results_2 = get_fuseki_data_json(query_2)
    if(len(results_2) > 0):
      for result in results_2:
        subject = result["s"]["value"]

      subject = "<" + subject + ">"
      object_query = """
          PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
          SELECT ?id ?value
          WHERE {
                graph """ + graph_name + """ {
                  ?s ?p pd3:Object;
                  pd3:output ?output;
                  pd3:id ?id;
                  pd3:value ?value.
              FILTER(?output = """ + subject + """)
              }
          }
      """
      object_result = get_fuseki_data_json(object_query)
      if(len(object_result) > 0):
        for result in object_result:
          object_id = result["id"]["value"]
          object_value = result["value"]["value"]

        return object_id, object_value

  return None, None

In [14]:
#------------------------------------------------------------------
# fusekiからデータを取得
#------------------------------------------------------------------
def get_fuseki_data_json(query):
  sparql = SPARQLWrapper("http://localhost:3030/akiyama/sparql")
  sparql.setQuery(query)

  sparql.setReturnFormat(JSON)
  results = sparql.query().convert()

  return results["results"]["bindings"]

In [15]:
#------------------------------------------------------------------
# fusekiにあるデータを更新 (削除、更新)
#------------------------------------------------------------------
def fuseki_update(query):
  sparql = SPARQLWrapper("http://localhost:3030/akiyama/update")

  sparql.setQuery(query) 
  sparql.method = 'POST'
  sparql.query()

  query_results = sparql.query()
  result = query_results.response.read().decode()

  # 削除成功
  if result.find("Success") > 0:
    return True
  # 削除失敗
  return False


In [16]:
#------------------------------------------------------------------
# グラフ画像を作成
#------------------------------------------------------------------
def set_image(image_path, nodes, edges, labels, select_node, node_postion):
  graph=nx.DiGraph()
  graph.clear()
  plt.clf()

  #頂点を設定
  graph.add_nodes_from(nodes)
  #辺を設定
  graph.add_edges_from(edges)

  color_map = []
  if not select_node:
    color_map = ['tab:blue']
  else:
    for node in graph:
        if node in select_node:
          color_map.append('orange')
        else:
          color_map.append('tab:blue')

  edge_color = [edge["color"] for edge in graph.edges.values()]

  #graph size
  plt.figure(3,figsize=(8,8)) 

  #plt.margins(0.5, 0)
  #node_size default=300
  nx.draw(graph, with_labels=False, node_color=color_map, font_family='IPAexGothic',\
        node_size=500, pos=node_postion, edge_color=edge_color, width=2.0, alpha=0.5)

  nx.draw_networkx_labels(graph, node_postion, labels, font_size=14, font_family='IPAexGothic',\
                          font_color='black')
  #保存
  plt.savefig(image_path)

In [17]:
#------------------------------------------------------------------
# member関連付け処理で追加するデータを用意
#------------------------------------------------------------------
def insert_member(graph_name, current_item, node_id, uri, member_uri):

  fuseki_insert_data = []
  member_node_list = []
  pd3 = Namespace('http://DigitalTriplet.net/2021/08/ontology#')

  #--------------------------------------------------------------
  # memberとmemberのlayerを取得
  #--------------------------------------------------------------
  member_query = """
      PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
      SELECT distinct ?node ?layer
      WHERE {
        graph """ + graph_name + """ {
          ?node ?p pd3:Action;
                pd3:layer ?layer
          filter(regex(str(?node),""" + '"' + member_uri + '"' + """)) 
          FILTER NOT EXISTS {?node pd3:attribution ?parent}
        }
      }
  """
  query_result = get_fuseki_data_json(member_query)
  for result in query_result:
    member_node_list.append(result["node"]["value"])
    layer = result["layer"]["value"]

  #--------------------------------------------------------------
  # containerを追加
  #--------------------------------------------------------------
  # container名
  container_name = ''.join(random.choice(string.ascii_letters + string.digits) for _ in range(22))

  # RDF.type
  fuseki_insert_data.append([URIRef(member_uri)+URIRef(container_name),\
                            RDF.type, pd3.Container])
  # containerType
  fuseki_insert_data.append([URIRef(member_uri)+URIRef(container_name),\
                            pd3.containerType, Literal('specialization')])
  # contraction
  fuseki_insert_data.append([URIRef(member_uri)+URIRef(container_name),\
                            pd3.contraction, URIRef(uri)+URIRef(node_id)])
  # id
  fuseki_insert_data.append([URIRef(member_uri)+URIRef(container_name),\
                            pd3.id, Literal(container_name)])
  # geoBoundingHeight
  fuseki_insert_data.append([URIRef(member_uri)+URIRef(container_name),\
                            pd3.geoBoundingHeight, Literal('200')])
  # geoBoundingWidth
  fuseki_insert_data.append([URIRef(member_uri)+URIRef(container_name),\
                            pd3.geoBoundingWidth, Literal('400')])
  # layer
  fuseki_insert_data.append([URIRef(member_uri)+URIRef(container_name),\
                            pd3.layer, Literal(layer)])
  # member
  for member in member_node_list:
    fuseki_insert_data.append([URIRef(member_uri)+URIRef(container_name),\
                              pd3.member, URIRef(member)])
  # output
  fuseki_insert_data.append([URIRef(member_uri)+URIRef(container_name),\
                            pd3.output, URIRef(uri)+URIRef(node_id)])
  # value
  fuseki_insert_data.append([URIRef(member_uri)+URIRef(container_name),\
                            pd3.value, Literal(current_item)])
  #--------------------------------------------------------------
  # expansionを追加
  #--------------------------------------------------------------
  fuseki_insert_data.append([URIRef(uri)+URIRef(node_id),\
                            pd3.expansion, URIRef(member_uri)+URIRef(container_name)])
  #--------------------------------------------------------------
  # memberのattributionを追加
  #--------------------------------------------------------------
  for member in member_node_list:
    fuseki_insert_data.append([URIRef(member), pd3.attribution, \
                              URIRef(member_uri)+URIRef(container_name)])
  #--------------------------------------------------------------
  # container flowを追加
  #--------------------------------------------------------------
  container_flow_name = ''.join(random.choice(string.ascii_letters + string.digits) for _ in range(22))
  # RDF.type
  fuseki_insert_data.append([URIRef(member_uri)+URIRef(container_flow_name),\
                            RDF.type, pd3.ContainerFlow])
  # arcType
  fuseki_insert_data.append([URIRef(member_uri)+URIRef(container_flow_name),\
                            pd3.arcType, Literal('hierarchization')])
  # id
  fuseki_insert_data.append([URIRef(member_uri)+URIRef(container_flow_name),\
                            pd3.id, Literal(container_flow_name)])
  # layer
  fuseki_insert_data.append([URIRef(member_uri)+URIRef(container_flow_name),\
                            pd3.layer, Literal(layer)])
  # source
  fuseki_insert_data.append([URIRef(member_uri)+URIRef(container_flow_name),\
                            pd3.source, URIRef(member_uri)+URIRef(container_name)])
  # target
  fuseki_insert_data.append([URIRef(member_uri)+URIRef(container_flow_name),\
                            pd3.target, URIRef(uri)+URIRef(node_id)])
  # value
  fuseki_insert_data.append([URIRef(member_uri)+URIRef(container_flow_name),\
                            pd3.value, Literal("")])

  if fuseki_insert(app.selected_graph_name, fuseki_insert_data):
    messagebox.showinfo('確認', 'Fusekiにインサートしました！')
  else:
    messagebox.showerror('Error', 'インサート失敗!')

In [18]:
#------------------------------------------------------------------
# member関連付け削除処理
#------------------------------------------------------------------
def delete_member(graph_name, node_uri, node_name, node_id, member_list, member_id_list):

  # コンテナidを取得
  query = """
      PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
      SELECT ?s ?id
      WHERE {
            graph """ + graph_name + """ {
              ?s ?p pd3:Container;
                pd3:id ?id;
                pd3:value ?value.
          FILTER(?value = """ + '"' + node_name + '"' + """)
          }
      }
  """
  query_result = get_fuseki_data_json(query)
  if(len(query_result) > 0):
    for result in query_result:
      con_id = result["id"]["value"]
      con_subject = result["s"]["value"]
      con_uri = con_subject.replace(con_id, "")

  # コンテナフロー情報を取得
  flow_query = """
      PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
      SELECT ?s ?id
      WHERE {
            graph """ + graph_name + """ {
              ?s ?p pd3:ContainerFlow;
                pd3:source ?source;
                pd3:id ?id;
                pd3:target ?target.
          FILTER(?source= """ + '<' + con_subject + '>' + """)
          FILTER(?target= """ + '<' + node_uri + node_id + '>' + """)
          }
      }
  """
  flow_result = get_fuseki_data_json(flow_query)
  if(len(flow_result) > 0):
    for result in flow_result:
      flow_subject = result["s"]["value"]
      flow_id = result["id"]["value"]

  # コンテナ削除クエリ
  con_query = """
      PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
      WITH """ + graph_name + """
      DELETE { ?s ?p ?o }
      WHERE { ?s ?p ?o ; 
              pd3:id """ + '"' + con_id + '"' + """ }
  """
  # 親のexpansion削除クエリ
  exp_query = """
      PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
      WITH """ + graph_name + """
      DELETE { ?s pd3:expansion """ + "<" + con_subject + ">" + """ }
      WHERE { ?s ?p ?o ; 
              pd3:id """ + '"' + node_id + '"' + """ }
  """
  # コンテナフロー削除クエリ
  flow_query = """
      PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
      WITH """ + graph_name + """
      DELETE { ?s ?p ?o }
      WHERE { ?s ?p ?o ; 
              pd3:id """ + '"' + flow_id + '"' + """ }
  """

  if fuseki_update(con_query) and fuseki_update(exp_query) and fuseki_update(flow_query):
    # コンテナのメンバーattribution削除クエリ
    for member_id in member_id_list:
      attr_query = """
          PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
          WITH """ + graph_name + """
          DELETE { ?s pd3:attribution """ + "<" + con_subject + ">" + """ }
          WHERE { ?s ?p ?o ; 
                  pd3:id """ + '"' + member_id + '"' + """ }
      """
      if fuseki_update(attr_query):
        continue
      else:
        messagebox.showerror('Error', '削除失敗!')
    messagebox.showinfo('確認', '削除しました！')
  else:
    messagebox.showerror('Error', '削除失敗!')

  # 親のuriとコンテナのuri一致する場合、uri入り替え処理
  if(node_uri == con_uri):
    print("uri処理")
    change_child_uri(graph_name, con_uri, member_list, member_id_list)

In [19]:
#------------------------------------------------------------------
# uri入り替え処理
#------------------------------------------------------------------
def change_child_uri(graph_name, uri, member_list, member_id_list):
  print("change_child_uri")

  container_count = get_ep_container_count(graph_name, uri)

  node_list = {}
  id_list = {}
  node_list['top'] = member_list
  id_list['top'] = member_id_list

  if(container_count != 0):
    (node_list, id_list) = get_member(graph_name, uri, container_count, False, node_list, id_list)

  # 新uriを生成
  new_uri = "http://" + ''.join(random.choice(string.ascii_letters + string.digits) for _ in range(10))
  # 新prefix
  new_prefix = "new-"+ ''.join(random.choice(string.ascii_letters + string.digits) for _ in range(10))


In [21]:
#------------------------------------------------------------------
# メンバーを取得
#------------------------------------------------------------------
def get_member(graph_name, uri, count, end, node_list, id_list):

  if(count == 0) or (end == True):
    return node_list, id_list

  node_list_copy = copy.copy(node_list)

  for key in node_list_copy.keys():
    for index, item in enumerate(node_list_copy[key]):
      query = """
          PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
          SELECT ?s
          WHERE {
                  graph """ + graph_name + """ {
                    ?s ?p pd3:Container;
                    pd3:value ?value.
                FILTER(?value = """ + '"' + item + '"'  + """ )
                FILTER(regex(str(?s),""" + '"' + uri + '"' + """)) 
                }
        }
      """
      query_result = get_fuseki_data_json(query)
      if(len(query_result) > 0):
        node_list[item] = []
        id_list[item] = []
        count -= 1
        for result in query_result:
          container = result["s"]["value"]

        query_2 = """
          PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
          SELECT ?id ?value
          WHERE {
                graph """ + graph_name + """ {
                  ?s ?p pd3:Action;
                  pd3:attribution ?parent;
                  pd3:id ?id;
                  pd3:value ?value.
              FILTER(?parent = <""" + container + """> )
              FILTER(regex(str(?s),""" + '"' + uri + '"' + """))
              }
        }
        """
        query_result = get_fuseki_data_json(query_2)
        for result in query_result:
          value = result["value"]["value"]
          id = result["id"]["value"]
          node_list[item].append(value)
          id_list[item].append(id)
          
      if (index+1 == len(id_list[key])):
          end = True
          return get_member(graph_name, uri, count, end, node_list, id_list)


In [22]:
#------------------------------------------------------------------
# use関連付け処理で追加するデータを用意
#------------------------------------------------------------------
def insert_use(graph_name, uri, node_id, log_graph, log_uri, log_node_id):
  fuseki_insert_data_log = []
  fuseki_insert_data_main = []
  pd3 = Namespace('http://DigitalTriplet.net/2021/08/ontology#')

  # ログのUse関連付けを追加
  fuseki_insert_data_log.append([URIRef(log_uri)+URIRef(log_node_id),\
                            pd3.use, URIRef(uri)+URIRef(node_id)])
  fuseki_insert(log_graph, fuseki_insert_data_log)

  # GPMのUseBy関連付けを追加
  fuseki_insert_data_main.append([URIRef(uri)+URIRef(node_id),\
                            pd3.useBy, URIRef(log_uri)+URIRef(log_node_id)])

  if fuseki_insert(log_graph, fuseki_insert_data_log) and \
      fuseki_insert(graph_name, fuseki_insert_data_main):
    messagebox.showinfo('確認', 'Fusekiにインサートしました!')
  else:
    messagebox.showerror('Error', 'インサート失敗!')

In [23]:
#------------------------------------------------------------------
# use関連付け削除処理
#------------------------------------------------------------------
def delete_use(graph_name, uri, node_id, log_graph, log_uri, log_node_id):

  delete_use_data = "<" + uri + node_id + ">"
  query_use = """
      PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
      WITH """ + log_graph + """
      DELETE { ?s pd3:use """ + delete_use_data + """ }
      WHERE { ?s ?p ?o ; 
              pd3:id """ + '"' + log_node_id + '"' + """ }
  """

  delete_useby_data = "<" + log_uri + log_node_id + ">"
  query_useby = """
      PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
      WITH """ + graph_name + """
      DELETE { ?s pd3:useBy """ + delete_useby_data + """ }
      WHERE { ?s ?p ?o ; 
              pd3:id """ + '"' + node_id + '"' + """ }
  """

  if fuseki_update(query_use) and fuseki_update(query_useby):
    messagebox.showinfo('確認', '削除しました！')
  else:
    messagebox.showerror('Error', '削除失敗!')


In [24]:
#------------------------------------------------------------------
# Fusekiにデータを追加
#------------------------------------------------------------------
def fuseki_insert(graph_name, insert_data):
  fuseki = FusekiUpdate('http://localhost:3030', 'akiyama')
  g = Graph()

  RdfUtils.add_list_to_graph(g, insert_data)
  spo_str = '\n'.join(
      [f'{s.n3()} {p.n3()} {o.n3()}.' for (s, p, o) in g]
  )

  fuseki_query = """
                PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
                INSERT DATA {
                  GRAPH """ + graph_name + "{" + spo_str + """} }"""
  query_results = fuseki.run_sparql(fuseki_query)

  result = query_results.response.read().decode()

  if result.find("Success") > 0:
    return True

  return False

In [25]:
#------------------------------------------------------------------
# Fusekiから単一グラフデータをエクスポート
#------------------------------------------------------------------
def fuseki_export():
  if (not app.selected_graph_name):
    messagebox.showerror('Error', 'No graph selected！')
  else:
    # ttlファイルをエクスポート
    ttl_file_path = 'fuseki_browser_export.ttl'
    graphml_file_path = 'fuseki_browser_export.ttl'
    query = """CONSTRUCT {?subject ?predicate ?object}
              WHERE {
                      graph """ + app.selected_graph_name + """ {
                      ?subject ?predicate ?object
                      }
                    }"""
    get_ttlfile_data(query, ttl_file_path)

    # graphmlファイルをエクスポート
    #g_nx = nx.DiGraph()
    #ttl_to_networkx(g_nx, ttl_file_path)
    #nx.write_graphml(g_nx, graphml_file_path)

    messagebox.showinfo('確認', 'エクスポートしました！')

In [26]:
#------------------------------------------------------------------
# Fusekiから全データをエクスポート
#------------------------------------------------------------------
def fuseki_export_all():
  if (not app.selected_graph_name):
    messagebox.showerror('Error', 'No graph selected！')
  else:
    file_path = 'fuseki_browser_export_all.ttl'
    query = """CONSTRUCT {?subject ?predicate ?object}
              WHERE {
                      graph ?gen {
                      ?subject ?predicate ?object
                      }
                    }"""
    get_ttlfile_data(query, file_path)
    messagebox.showinfo('確認', 'ttlファイルをエクスポートしました！')

In [27]:
if __name__ == "__main__":
  # Windowを生成する。
  root = tk.Tk()
  app = Application(master=root)
  # Windowをループさせて、継続的にWindow表示させる。
  app.mainloop()